In [107]:
# IMPORTING LIBRARIES 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier 
import sklearn.datasets
import sklearn.preprocessing
import sklearn.random_projection
import sklearn.neighbors
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [108]:
# starting with these parameters
n = 5
profit_taking = 0.0025
p = 0.64

In [109]:
#READ IN DATA
df_original = pd.read_csv("data/final_clean_FX_data.csv")

In [110]:
df_original = df_original.sort_values(by =  "Date", ascending=True)
df_original["range"] = (df_original["High"] - df_original["Low"]) / df_original["Open"]
df = df_original
df["c_over_o"] = (df["Close"] - df["Open"]) / df["Open"]
df["h_over_o"] = (df["High"]  - df["Open"]) / df["Open"]
df["l_over_o"] = (df["Low"]   - df["Open"]) / df["Open"]

In [111]:
# CREATE PRIOR n DAYS FEATURE
for before in range(1, n+1):
    df[f"Close_{before}_before"] = df["Close"].shift(before)
    df[f"Open_{before}_before"] = df["Open"].shift(before)
    df[f"High_{before}_before"] = df["High"].shift(before)
    df[f"Low_{before}_before"] = df["Low"].shift(before)
    df[f"c_over_o_lag{before}"] = df["c_over_o"].shift(before)
    df[f"h_over_o_lag{before}"] = df["h_over_o"].shift(before)
    df[f"l_over_o_lag{before}"] = df["l_over_o"].shift(before)



In [112]:
#feature enginnering
df["ret_1d"] = np.log(df["Close"]) - np.log(df["Close_1_before"])
df["ret_5d"] = np.log(df["Close"]) - np.log(df["Close_5_before"])
df["vol_5d"]  = df["ret_1d"].rolling(5).std()
df["vol_10d"] = df["ret_1d"].rolling(10).std()
df["mom_5d"] = df["ret_1d"].rolling(5).mean()
df["mom_10d"] = df["ret_1d"].rolling(10).mean()
df["range_5d"] = df["range"].rolling(5).mean()

feature_cols = [
    "ret_1d", "ret_5d",
    "vol_5d", "vol_10d",
    "range", "mom_5d", 
    "mom_10d", "range_5d"
]

df[feature_cols] = df[feature_cols].shift(1)

df["target"] = (df["High"] > (profit_taking+1)*df["Open"]).astype(int)
target = df[["target"]]
df = df.dropna()

In [113]:
train_df   = df[(df['Date'] >= '2017-01-01') & (df['Date'] < '2020-01-01')]
val_df   = df[(df['Date'] >= '2020-01-01') & (df['Date'] < '2022-01-01')]
test_df  = df[df['Date'] >= '2022-01-01']

x_train = train_df.drop(columns=['target', 'Date', 'Close', 'Open', 'High', 'Low'])
y_train = train_df['target']

x_val = val_df.drop(columns=['target', 'Date', 'Close', 'Open', 'High', 'Low'])
y_val = val_df['target']

x_test = test_df.drop(columns=['target', 'Date', 'Close', 'Open', 'High', 'Low'])
y_test = test_df['target']

In [114]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

(782, 52)
(523, 52)
(1011, 52)
target
1    0.604859
0    0.395141
Name: proportion, dtype: float64
target
1    0.506692
0    0.493308
Name: proportion, dtype: float64
target
0    0.551929
1    0.448071
Name: proportion, dtype: float64


In [115]:
# Scaling the data

standardize = StandardScaler(with_mean=True, with_std=True)
standardize.fit(x_train)
x_train_scaled = pd.DataFrame(
    standardize.transform(x_train),
    columns=x_train.columns,
    index=x_train.index
)
x_val_scaled = pd.DataFrame(
    standardize.transform(x_val),
    columns=x_val.columns,
    index=x_val.index
)
x_test_scaled = pd.DataFrame(
    standardize.transform(x_test),
    columns=x_test.columns,
    index=x_test.index
)

In [116]:
# MODEL TRAINING
param_grid = {
    "hidden_layer_sizes": [
        (16,),
        (24,),
        (24, 10),
        (32, 16),
        (32, 16, 8)
    ],
    "alpha": [1e-5, 1e-4, 1e-3],
    "activation": ["relu", "tanh"]
}


mlp = MLPClassifier(
    max_iter=1000,
    random_state=42
)

grid = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    scoring="accuracy",     # validation accuracy
    cv=3,                   # internal CV on training set
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)
grid.fit(x_train, y_train)

print("Best parameters:")
print(grid.best_params_)

best_model = grid.best_estimator_

train_accuracy = best_model.score(x_train, y_train)
val_accuracy   = best_model.score(x_val, y_val)

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Validation accuracy: {val_accuracy:.4f}")

#model1.fit(x_train, y_train)

#validation_accuracy = model1.score(x_val, y_val)
#print(f"validation_accuracy1={validation_accuracy:0.4f}")
#train_accuracy = model1.score(x_train, y_train)
#print(f"train_accuracy1={train_accuracy:0.4f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(16,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(16,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(16,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24,); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24, 10); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(32, 16); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24, 10); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(24, 10); total time=   0.0s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(32, 16); total time=   0.0s


In [118]:
model1 = MLPClassifier(
    hidden_layer_sizes=[16,],
    activation="tanh",
    alpha=1e-05,
    max_iter=1000,
    random_state = 42
)

model1.fit(x_train, y_train)

validation_accuracy = model1.score(x_val, y_val)
print(f"validation_accuracy1={validation_accuracy:0.4f}")
train_accuracy = model1.score(x_train, y_train)
print(f"train_accuracy1={train_accuracy:0.4f}")

validation_accuracy1=0.5067
train_accuracy1=0.6049


In [119]:
if True:
    test_accuracy = model1.score(x_test, y_test)
    print(f"test_accuracy={test_accuracy}")

    y_true = y_test.values.ravel()
    probs_test = model1.predict_proba(x_test)[:, 1]
    preds_test = model1.predict(x_test)

test_accuracy=0.44510385756676557


In [120]:
#if True:
  #  total_return = df_evaluation["pnl"].sum()
  #  num_trades = df_evaluation["enter"].sum()
  #  hit_rate = df_evaluation.loc[df_evaluation["enter"] == 1, "target"].mean()
  #  avg_return_per_trade = df_evaluation.loc[df_evaluation["enter"] == 1, "pnl"].mean()

    #print("Total return:", total_return)
  #  print("Number of trades:", num_trades)
  #  print("Hit rate:", hit_rate)
  #  print("Avg return per trade:", avg_return_per_trade)

In [121]:
#df_evaluation['prob'].describe()

In [122]:
#eval NN

## cleaning data
X_test = test_df.drop(columns=["target", "Date", "Close", "Open", "High", "Low"])
y_test = test_df["target"]
X_test_scaled = standardize.transform(X_test)

## probabilities
probs = model1.predict_proba(X_test_scaled)[:, 1]

# probability distribution
print("Probability summary:")
print(pd.Series(probs).describe())

# ROC-accuracy
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, probs)
print(f"ROC-AUC: {auc:.3f}")


# Quintile hit-rate 
df_eval = test_df.copy()
df_eval["prob"] = probs

df_eval["quintile"] = pd.qcut(df_eval["prob"], 5, labels=False)

quintile_perf = (
    df_eval
    .groupby("quintile")["target"]
    .mean()
)

print("TP hit-rate by probability quintile:")
print(quintile_perf)

# threshold-based trading behavior
df_eval["enter"] = (df_eval["prob"] >= p).astype(int)

num_trades = df_eval["enter"].sum()
trade_hit_rate = df_eval.loc[df_eval["enter"] == 1, "target"].mean()
base_hit_rate = df_eval["target"].mean()

print(f"Base hit rate: {base_hit_rate:.3f}")
print(f"Trades taken: {num_trades}")
print(f"Hit rate on trades: {trade_hit_rate:.3f}")


precision = precision_score(
    y_test,
    df_eval["enter"]
)

print(f"Precision (hit rate on entered days): {precision:.3f}")

coverage = df_eval["enter"].mean()

print(f"Coverage (fraction of days entered): {coverage:.3f}")

# -------------------------
# Model-based cumulative equity
# -------------------------
df_pnl = df_eval.copy()

# open-to-close return
df_pnl["ret_oc"] = (df_pnl["Close"] - df_pnl["Open"]) / df_pnl["Open"]

df_pnl["daily_return"] = 0.0

# TP hit → take profit
df_pnl.loc[
    (df_pnl["enter"] == 1) & (df_pnl["target"] == 1),
    "daily_return"
] = profit_taking

# TP not hit → close-to-open P&L
df_pnl.loc[
    (df_pnl["enter"] == 1) & (df_pnl["target"] == 0),
    "daily_return"
] = df_pnl["ret_oc"]

# compound equity
df_pnl["equity_model"] = (1 + df_pnl["daily_return"]).cumprod()

model_final_equity = df_pnl["equity_model"].iloc[-1]
print(f"Final equity (model strategy): {model_final_equity:.3f}")


df_base = df_eval.copy()

df_base["ret_oc"] = (df_base["Close"] - df_base["Open"]) / df_base["Open"]

df_base["daily_return"] = df_base["ret_oc"]

# If TP hit, override with TP gain
df_base.loc[
    df_base["target"] == 1,
    "daily_return"
] = profit_taking

df_base["equity_base"] = (1 + df_base["daily_return"]).cumprod()

base_final_equity = df_base["equity_base"].iloc[-1]
print(f"Final equity (enter every day): {base_final_equity:.3f}")

relative_performance = model_final_equity / base_final_equity
print(f"Relative performance (model / baseline): {relative_performance:.3f}x")

Probability summary:
count    1011.000000
mean        0.215889
std         0.084894
min         0.071253
25%         0.163787
50%         0.191365
75%         0.235149
max         0.743733
dtype: float64
ROC-AUC: 0.592
TP hit-rate by probability quintile:
quintile
0    0.344828
1    0.391089
2    0.470297
3    0.480198
4    0.554455
Name: target, dtype: float64
Base hit rate: 0.448
Trades taken: 3
Hit rate on trades: 1.000
Precision (hit rate on entered days): 1.000
Coverage (fraction of days entered): 0.003
Final equity (model strategy): 1.008
Final equity (enter every day): 1.042
Relative performance (model / baseline): 0.967x


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
